In [1]:
import pandas as pd
import utils

In [2]:
pair = "EUR_USD"
granularity = "H1"

In [10]:
pair = "EUR_USD"
granularity = "H1"


In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [7]:
non_cols = ['time', 'volume']

In [8]:
non_cols

['time', 'volume']

In [11]:
mod_cols = [x for x in df.columns if x not in non_cols]

In [13]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [15]:
df.describe()

,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,3575.061000,1.082658,1.083296,1.082024,1.082653,1.082567,1.083215,1.081935,1.082571,1.082749,1.083384,1.082106,1.082735
std,2449.141788,0.017391,0.017399,0.017371,0.017390,0.017390,0.017399,0.017370,0.017389,0.017393,0.017400,0.017372,0.017391
min,127.000000,1.045260,1.046640,1.044850,1.045270,1.045180,1.046550,1.044770,1.045200,1.045330,1.046720,1.044930,1.045340
25%,1955.500000,1.069708,1.070120,1.069143,1.069675,1.069605,1.070048,1.069040,1.069605,1.069787,1.070225,1.069245,1.069755
50%,3066.000000,1.085000,1.085580,1.084370,1.084970,1.084910,1.085500,1.084265,1.084880,1.085105,1.085655,1.084450,1.085060
75%,4523.250000,1.095432,1.096045,1.094790,1.095422,1.095333,1.095975,1.094702,1.095310,1.095550,1.096140,1.094873,1.095515
max,19273.000000,1.125740,1.127570,1.125050,1.125760,1.125650,1.127490,1.124960,1.125670,1.125820,1.127650,1.125130,1.125840
